In [16]:
import numpy as np
import pandas as pd

from utils.common import *

C:\Users\vesely\AppData\Local\conda\conda\envs\crypto\lib\site-packages\matplotlib\cbook\deprecation.py:107: MatplotlibDeprecationWarning: The finance module has been deprecated in mpl 2.0 and will be removed in mpl 2.2. Please use the module mpl_finance instead.


# Get Data

In [17]:
import urllib.request
import json

def retrieve(url):
    with urllib.request.urlopen(url) as response:
        html = response.read().decode("utf-8")
        data = json.loads(html)
    return data

In [18]:
def queryPoloniex(beginDatetime, endDatetime, currencyPair, period):
    beginTimestamp, endTimestamp = beginDatetime.timestamp(), endDatetime.timestamp()
    url = 'https://poloniex.com/public?command=returnTradeHistory&currencyPair={pair}&start={begin}&end={end}'
#     url = 'https://poloniex.com/public?command=returnChartData&currencyPair={pair}&start={begin}&end={end}&period={period}'
    return retrieve(url.format(pair=currencyPair, begin=beginTimestamp, end=endTimestamp))

In [19]:
from dateutil.rrule import rrule, YEARLY
from dateutil.relativedelta import relativedelta
from time import sleep

def retrieveLongWindowPoloniex(beginTime, endTime, currencyPair, period):
    beginDT = strTimeToDatetime(beginTime)
    endDT = strTimeToDatetime(endTime)
    data = []
    for dt in rrule(YEARLY, dtstart=beginDT, until=endDT):
        dt2 = dt+relativedelta(years=1)
        data += queryPoloniex(dt, dt+relativedelta(years=1), currencyPair, period)
        sleep(1)
    
    return data
        

In [20]:
def renameDateToTimestamp(df):
    return df.rename(columns={'date': 'timestamp'})

In [21]:
def poloniexDataToDf(data):
    df = pd.DataFrame(data)
    df = renameDateToTimestamp(df)
    df['date'] = df['timestamp'].apply(timestampToDate)
    return df.drop_duplicates()

# Download

In [22]:
def retrieveAndSave(cPair, startDateTime, endDateTime, interval, intervalStr):
    startDate, endDate = startDateTime[:10], endDateTime[:10]
    data = retrieveLongWindowPoloniex(startDateTime, endDateTime, cPair, interval)
    df = poloniexDataToDf(data)
    fileName = '{cPair}_{intervalStr}_{startDate}_{endDate}.tsv'.format(**locals())
    dataRoot = 'D:\\Dropbox\\My work\\krypl-project\\'
    path = '{dataRoot}\\data\\poloniex\\{intervalStr}\\{fileName}'.format(**locals())
    writeTsv(df, path)

In [12]:
startDateTime, endDateTime = '2014-01-01 00:00:00', '2018-03-27 23:59:59'
interval, intervalStr = 300, '5min'
cPairs = [
    'USDT_BCH', 'BTC_BCH', 
    'BTC_XEM', 
    'USDT_STR', 'BTC_STR',
    'BTC_DASH',
    'USDT_ETC', 'BTC_ETC',
    'BTC_LSK'
]
cPairs = ['USDT_BTC']

In [38]:
for cPair in cPairs:
    print(cPair, end='\t')
    retrieveAndSave(cPair, startDateTime, endDateTime, interval, intervalStr)
    print('done')

USDT_BTC	done


In [32]:
tmp = queryPoloniex(strTimeToDatetime('2018-03-26 23:59:59'), strTimeToDatetime('2018-03-27 03:05:17'), 'USDT_BTC', None)

In [33]:
pd.DataFrame(tmp).date.min()

'2018-03-26 22:00:03'

In [34]:
pd.DataFrame(tmp).date.max()

'2018-03-27 01:05:12'